In [25]:
from datasets import load_dataset
import torch

DEVICE = torch.device("cuda" if torch.cuda.is_available() else "mps" if torch.backends.mps.is_available() else "cpu")

ds = load_dataset("wmt/wmt14", "de-en", cache_dir="./data/wmt14")

vocab_size = 30_000
vocab_path = "./data/bpe_tokenizer.json"

training_samples = 10_000
batch_size = 64
dataset_max_sample_len = 100

# bpe_v3_ep12
d_model=256
nhead=8
num_encoder_layers=4
num_decoder_layers=4
dim_feedforward=1024
dropout=0.1
max_len=150

# big_3.8770loss
# d_model=512
# nhead=8
# num_encoder_layers=2
# num_decoder_layers=2
# dim_feedforward=2048
# dropout=0.1
# max_len=150

# training
num_epochs = 100
warmup_steps = 2000
eval_iters = 30
patience = 30

label_smoothing = 0.1

# optimizer
start_lr = 3e-4
betas = (0.9, 0.98)
epsilon = 1e-9

In [2]:
from tokenizers import Tokenizer as HFTokenizer, decoders
from tokenizers.models import BPE
from tokenizers.trainers import BpeTrainer
from tokenizers.pre_tokenizers import Metaspace
from tokenization_vocab import HFTokenizerWrapper, Tokenizer
from pathlib import Path

bpe_tokenizer = HFTokenizer(BPE(unk_token=Tokenizer.UNK_TOKEN))
trainer = BpeTrainer(
    special_tokens=[Tokenizer.PAD_TOKEN, Tokenizer.SOS_TOKEN, Tokenizer.EOS_TOKEN, Tokenizer.UNK_TOKEN],
    vocab_size=vocab_size,
    show_progress=True
)

bpe_tokenizer.pre_tokenizer = Metaspace()
bpe_tokenizer.decoder = decoders.Metaspace()

pretrained = True  # Set to True if you want to load a previously saved tokenizer

Path(vocab_path).parent.mkdir(parents=True, exist_ok=True)

if Path(vocab_path).is_file():
    pretrained = True

if pretrained:
    bpe_tokenizer = HFTokenizer.from_file(vocab_path)
else:
    bpe_tokenizer.train(
        [
            './datasets/wmt14_translate_de-en_test.csv',
            './datasets/wmt14_translate_de-en_train.csv',
            './datasets/wmt14_translate_de-en_validation.csv',
        ],
        trainer=trainer
    )

    bpe_tokenizer.save(vocab_path)


tokenizer = HFTokenizerWrapper(bpe_tokenizer)

In [4]:
from parallel_corpus import TranslationDataset, DataLoaderFactory
from tokenization_vocab import HFTokenizerWrapper

train_pairs = [(s['de'], s['en']) for s in ds['train'][:training_samples]['translation']]
test_pairs = [(s['de'], s['en']) for s in ds['test']['translation']]

sample_sentences_de = [s[0] for s in train_pairs]
sample_sentences_en = [s[1] for s in train_pairs]

tokenizer = HFTokenizerWrapper(bpe_tokenizer)

train_sents, test_sents = train_pairs, test_pairs

train_ds = TranslationDataset(
    source_sentences=[s[0] for s in train_sents],
    target_sentences=[s[1] for s in train_sents],
    source_tokenizer=tokenizer,
    target_tokenizer=tokenizer,
    max_length=dataset_max_sample_len
)

test_ds = TranslationDataset(
    source_sentences=[s[0] for s in test_sents],
    target_sentences=[s[1] for s in test_sents],
    source_tokenizer=tokenizer,
    target_tokenizer=tokenizer,
    max_length=dataset_max_sample_len
)

train_loader = DataLoaderFactory.create_dataloader(
    dataset=train_ds,
    batch_size=batch_size,
    pad_idx=tokenizer.pad_idx,
    shuffle=True  # IMPORTANT: Shuffle data each epoch for better training
)

test_loader = DataLoaderFactory.create_dataloader(
    dataset=test_ds,
    batch_size=batch_size,
    pad_idx=tokenizer.pad_idx,
    shuffle=True  # IMPORTANT: Shuffle data each epoch for better training
)

Preprocessing dataset...
Preprocessed 9932 sentence pairs
Skipped 68 pairs (too long)
Preprocessing dataset...
Preprocessed 3002 sentence pairs
Skipped 1 pairs (too long)


In [26]:
# Import the TranslationTransformer
from translation_transformer import TranslationTransformer, TranslationTransformerPytorch

# Initialize the model with larger max_len to handle max_length + special tokens
model = TranslationTransformer(
    src_vocab_size=len(tokenizer),
    tgt_vocab_size=len(tokenizer),
    d_model=d_model,
    nhead=nhead,
    num_encoder_layers=num_encoder_layers,
    num_decoder_layers=num_decoder_layers,
    dim_feedforward=dim_feedforward,
    dropout=dropout,
    max_len=max_len,
    padding_idx=tokenizer.pad_idx
)

print(f"Model initialized!")
print(f"Total parameters: {sum(p.numel() for p in model.parameters()):,}")

Model initialized!
Total parameters: 30,431,536


In [27]:
state_dict = torch.load("./models/bpe_v3_ep12_3.8523loss.pt", map_location=DEVICE)['model_state_dict']
new_state_dict = {
    k.replace("_orig_mod.", ""): v
    for k, v in state_dict.items()
}
model.load_state_dict(new_state_dict)
model.eval()

TranslationTransformer(
  (src_embedding): WordEmbedding(
    (embedding): Embedding(30000, 256, padding_idx=0)
  )
  (tgt_embedding): WordEmbedding(
    (embedding): Embedding(30000, 256, padding_idx=0)
  )
  (positional_encoding): PositionalEncoding(
    (dropout): Dropout(p=0.1, inplace=False)
  )
  (encoder_layers): ModuleList(
    (0-3): 4 x TransformerEncoderLayer(
      (self_attn): MultiHeadAttention(
        (qkv_proj): Linear(in_features=256, out_features=768, bias=False)
        (out_proj): Linear(in_features=256, out_features=256, bias=False)
      )
      (norm1): LayerNorm((256,), eps=1e-05, elementwise_affine=True)
      (norm2): LayerNorm((256,), eps=1e-05, elementwise_affine=True)
      (ff): Sequential(
        (0): Linear(in_features=256, out_features=1024, bias=True)
        (1): ReLU()
        (2): Dropout(p=0.1, inplace=False)
        (3): Linear(in_features=1024, out_features=256, bias=True)
      )
      (dropout1): Dropout(p=0.1, inplace=False)
      (dropout2)

In [29]:
# Apply PyTorch optimizations
import torch

# 1. Enable TF32 for faster matmul on Ampere+ GPUs (A100, RTX 3090, etc.)
# This provides ~2x speedup for matrix multiplications with minimal accuracy loss
torch.set_float32_matmul_precision('high')  # Options: 'highest', 'high', 'medium'

# 2. For MPS (Apple Silicon), ensure we're using optimal settings
if DEVICE.type == "mps":
    # MPS backend is already optimized, but we can ensure memory efficiency
    torch.mps.empty_cache()  # Clear any cached memory
elif DEVICE.type == "cuda":
    # Enable TF32 for cuDNN convolutions as well
    torch.backends.cudnn.allow_tf32 = True
else:
    print("✓ Running on CPU (no GPU optimizations)")

# 3. Don't compile for now while debugging mask issues
# Once stable, uncomment below for 2-10x speedup
model_compiled = model
# model_compiled = torch.compile(model, mode='default')  # Options: 'default', 'reduce-overhead', 'max-autotune'

# Move model to device (GPU if available)
model = model.to(DEVICE)
model.train()

print(f"Using device: {DEVICE}")
print(f"Model moved to {DEVICE}")

Using device: mps
Model moved to mps


In [ ]:
import torch.nn as nn
import torch.optim as optim
from train import train

# Loss function and optimizer
criterion = nn.CrossEntropyLoss(ignore_index=tokenizer.pad_idx, label_smoothing=label_smoothing)
optimizer = optim.Adam(model.parameters(), lr=start_lr, betas=betas, eps=epsilon)

# Training
train_losses, best_loss = train(
    model=model_compiled,
    dataloader=train_loader,
    dataset_size=len(train_ds),
    train_loader=train_loader,
    test_loader=test_loader,
    criterion=criterion,
    optimizer=optimizer,
    device=DEVICE,
    num_epochs=num_epochs,
    warmup_steps=warmup_steps,
    eval_iters=eval_iters,
    patience=patience
)

Using device: mps
Model moved to mps
Starting training for 100 epochs...
Total batches per epoch: 140
Dataset size: 8938 samples
Learning rate: 2.9999999999999997e-05 (with warmup and decay)

Epoch 1/100


/opt/homebrew/lib/python3.13/site-packages/torch/utils/data/dataloader.py:692: UserWarning: 'pin_memory' argument is set as true but not supported on MPS now, device pinned memory won't be used.
  warnings.warn(warn_msg)


KeyboardInterrupt: 

In [ ]:
# Plot training progress
import matplotlib.pyplot as plt

plt.figure(figsize=(10, 6))
plt.plot(range(1, len(train_losses) + 1), train_losses, marker='o', linewidth=2, markersize=6)
plt.xlabel('Epoch', fontsize=12)
plt.ylabel('Training Loss', fontsize=12)
plt.title('Training Loss Over Time', fontsize=14, fontweight='bold')
plt.grid(True, alpha=0.3)
plt.tight_layout()
plt.show()

print(f"\nTraining Summary:")
print(f"  Initial Loss: {train_losses[0]:.4f}")
print(f"  Final Loss: {train_losses[-1]:.4f}")
print(f"  Best Loss: {best_loss:.4f}")
print(f"  Improvement: {((train_losses[0] - train_losses[-1]) / train_losses[0] * 100):.1f}%")

In [30]:
# Test translation on sample input
import torch
import sacrebleu

@torch.no_grad()
def translate_sample(sentence: str, model: TranslationTransformer,
                     src_tokenizer: HFTokenizerWrapper, tgt_tokenizer: HFTokenizerWrapper, max_len=100, device=DEVICE):
    """
    Translate a single sentence using the model with autoregressive generation.
    
    Args:
        sentence: Input sentence to translate
        model: TranslationTransformer model
        src_tokenizer: HFTokenizerWrapper for the source language
        tgt_tokenizer: HFTokenizerWrapper for the target language
        max_len: Maximum sequence length to generate
        device: Device to run on
        
    Returns:
        Translated sentence and token indices
    """
    model.eval()
    
    # Tokenize input
    src_tokens = src_tokenizer.tokenize(sentence)
    print(f"Input tokens: {src_tokens}")
    
    # Encode with EOS token only (source side)
    src_indices = src_tokenizer.encode(src_tokens, add_sos=False, add_eos=True)
    src_tensor = torch.tensor(src_indices, dtype=torch.long).unsqueeze(0).to(device)  # [1, seq_len]
    
    # Create source padding mask (all False since no padding in single sentence)
    src_key_padding_mask = torch.zeros(1, src_tensor.size(1), dtype=torch.bool).to(device)
    
    print(f"Input tensor shape: {src_tensor.shape}")
    print(f"Input indices: {src_indices}")
    
    # Initialize target with just SOS token
    tgt_indices = [tgt_tokenizer.sos_idx]
    
    # Autoregressive generation loop
    for _ in range(max_len):
        # Convert current target indices to tensor
        tgt_tensor = torch.tensor([tgt_indices], dtype=torch.long).to(device)  # [1, current_len]
        
        # Create target padding mask (all False since we're only generating, no padding)
        tgt_key_padding_mask = torch.zeros(1, tgt_tensor.size(1), dtype=torch.bool).to(device)
        
        # Forward pass with masks
        output = model(src_tensor, tgt_tensor, 
                      src_key_padding_mask=src_key_padding_mask,
                      tgt_key_padding_mask=tgt_key_padding_mask)  # [1, current_len, vocab_size]
        
        # Get prediction for the last token
        next_token_logits = output[0, -1, :]  # [vocab_size]
        next_token = torch.argmax(next_token_logits).item()
        
        # Append predicted token
        tgt_indices.append(next_token)
        
        # Stop if we predict EOS token
        if next_token == tgt_tokenizer.eos_idx:
            break
    
    print(f"Generated {len(tgt_indices)} tokens")
    print(f"Predicted indices: {tgt_indices}")
    
    # Decode back to tokens (skip SOS and EOS)
    translation = tgt_tokenizer.decode_to_text(tgt_indices)  # Remove SOS and EOS
        
    return translation, tgt_indices

# Test with a sample German sentence from the dataset
total_bleu = 0.0
print_enabled = False

batch_de, batch_en, _, _ = next(iter(test_loader))
samples = len(batch_de)

for idx, (de, en) in enumerate(zip(batch_de, batch_en)):
    sample_de = tokenizer.decode_to_text(de.tolist())
    sample_en = tokenizer.decode_to_text(en.tolist())

    translation, _ = translate_sample(
        sample_de, 
        model, 
        src_tokenizer=tokenizer,
        tgt_tokenizer=tokenizer,
        max_len=max_len,
        device=DEVICE
    )
    BLEUscore = sacrebleu.corpus_bleu([translation], [[sample_en]])
    total_bleu += BLEUscore.score

    if print_enabled:
        print(f"Original German: {sample_de}")
        print(f"Original English: {sample_en}")
        print("\n" + "="*60 + "\n")

        print("\n" + "="*60)
        print(f"\nModel Translation: '{translation}'")
        print(f"Reference Translation: {sample_en}")
        print(f"BLEU Score: {BLEUscore.score:.4f}")

print(f"\nAverage BLEU Score over {samples} samples: {(total_bleu / samples):.4f}")

Input tokens: ['▁Jahres', 'zeiten', 'bed', 'ingt', '▁laufen', '▁seit', '▁September', '▁auch', '▁wieder', '▁Stellen', 'angebote', '▁für', '▁Personal', '▁im', '▁Hotel', '-', '▁und', '▁Gast', 'stätten', 'ge', 'werbe', '▁ein', '.']
Input tensor shape: torch.Size([1, 24])
Input indices: [7466, 12056, 5937, 8964, 19415, 5667, 7363, 4078, 5045, 12539, 23277, 3944, 9891, 3928, 4419, 19, 3830, 10712, 24150, 3813, 29336, 3840, 20, 2]
Generated 32 tokens
Predicted indices: [1, 4642, 3790, 4950, 3815, 7363, 4283, 4111, 4328, 3774, 5232, 3815, 4645, 4439, 4040, 4328, 3796, 6784, 3815, 18472, 3796, 3790, 6029, 3815, 3790, 5212, 3822, 10026, 4317, 6449, 20, 2]
Input tokens: ['▁Im', '▁Anschluss', '▁an', '▁Ch', 'ens', '▁Entsch', 'uld', 'igung', '▁veröffentlich', 'te', '▁der', '▁Neue', '▁Express', '▁eine', '▁Entsch', 'uld', 'igung', '▁auf', '▁der', '▁Tit', 'els', 'eite', '▁und', '▁erklär', 'te,', '▁man', '▁habe', '▁es', '▁vers', 'äum', 't,', '▁die', '▁Berichte', '▁korrekt', '▁zu', '▁überprüfen', '.']
In

In [24]:

translation, _ = translate_sample(
    tokenizer.decode_to_text(test_ds[0][0].tolist()),
    model, 
    src_tokenizer=tokenizer,
    tgt_tokenizer=tokenizer,
    max_len=max_len,
    device=DEVICE
)
translation

Input tokens: ['▁Gut', 'ach', ':', '▁Noch', '▁mehr', '▁Sicherheit', '▁für', '▁Fuß', 'gänger']
Input tensor shape: torch.Size([1, 10])
Input indices: [15332, 3922, 32, 27488, 4479, 5427, 3944, 8804, 18100, 2]
Generated 13 tokens
Predicted indices: [1, 7418, 7429, 4323, 6114, 3838, 5397, 3774, 5879, 3887, 18864, 20, 2]


'And yet more security is still a matter for football.'


## Saving and loading the model

To load the model later, use:
```python
checkpoint = torch.load('./models/translation_model.pt')
model.load_state_dict(checkpoint['model_state_dict'])
optimizer.load_state_dict(checkpoint['optimizer_state_dict'])
```